In [ ]:
# ! sudo pip install snowflake-ml-python==1.0.2 -U 
# ! pip install snowflake-snowpark-python==1.5.1 -U
# ! pip install --upgrade xgboost==1.7.3

In [1]:
from snowflake.ml.modeling.pipeline import Pipeline
from snowflake.ml.modeling.xgboost import XGBClassifier
from snowflake.ml.modeling.linear_model import SGDClassifier
from snowflake.ml.modeling.preprocessing import MinMaxScaler, OrdinalEncoder, OneHotEncoder
from sklearn.metrics import mean_absolute_percentage_error

In [2]:
from snowflake.snowpark import Session
import configparser
def get_session():
    parser = configparser.ConfigParser()
    # Add the credential file name here
    parser.read('/notebooks/notebooks/config.ini')

    connection_params = dict(user=parser['Credentials']['user'], 
                         password=parser['Credentials']['password'], 
                         account=parser['Credentials']['account'], 
                         warehouse=parser['Credentials']['warehouse'], 
                         database=parser['Credentials']['database'],
                         schema=parser['Credentials']['schema'], 
                         role=parser['Credentials']['role'])

    session = Session.builder.configs(connection_params).create()
    return session



In [3]:
session = get_session()

### Load the final data from snowflake table
The dataset is prepared at this point of time and is available in snowflake inside a table "predictive_maintenance_final"

In [4]:
table_name = 'predictive_maintenance_final'

In [5]:
sf_df = session.table(table_name).drop('ROW')

In [6]:
sf_df.show(3)

-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"FAILURE"  |"METRIC5"  |"METRIC6"  |"DAY"  |"DAY_WEEK"  |"SECTOR"  |"OP_PERIOD"  |"DEV_RECONNECTED"  |"MNW1"    |"DIF_M6"  |"DIF_M5"  |"LOG_M2"           |"LOG_M3"            |"LOG_M4"            |"LOG_M7"  |"LOG_M9"           |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|0          |6          |407438     |1      |3           |S1F0      |1            |0                  |26953834  |0         |0         |2.079441541679836  |0.0                 |3.9702919135521215  |0.0       |2.079441541679836  |
|0          |6          |403174     |1      |3           |S1F0      |1          

Prepapre the columns for transformation

In [7]:
CATEGORICAL_COLUMNS = ["SECTOR"]
CATEGORICAL_COLUMNS_OE = ["SECTOR_OE"]
NUMERICAL_COLUMNS = ['METRIC5','METRIC6','DAY','DAY_WEEK','OP_PERIOD','MNW1','DIF_M6','DIF_M5','LOG_M2',
 'LOG_M3','LOG_M4','LOG_M7','LOG_M9']
LABEL_COLUMNS = ["FAILURE"]
OUTPUT_COLUMNS = ["PREDICTION"]

Splitting the snowpark dataframe creates a temp table inside the account

In [8]:
train_df, test_df = sf_df.random_split([0.8,0.2])

In [9]:
train_df.show(3)

-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"FAILURE"  |"METRIC5"  |"METRIC6"  |"DAY"  |"DAY_WEEK"  |"SECTOR"  |"OP_PERIOD"  |"DEV_RECONNECTED"  |"MNW1"    |"DIF_M6"  |"DIF_M5"  |"LOG_M2"           |"LOG_M3"            |"LOG_M4"            |"LOG_M7"  |"LOG_M9"           |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|0          |6          |407438     |1      |3           |S1F0      |1            |0                  |26953834  |0         |0         |2.079441541679836  |0.0                 |3.9702919135521215  |0.0       |2.079441541679836  |
|0          |6          |403174     |1      |3           |S1F0      |1          

Define the pipeline with preprocessing steps and model definition

In [10]:
pipeline = Pipeline(
    steps=[
            (
                "OHE",
                OrdinalEncoder(
                input_cols=CATEGORICAL_COLUMNS,
                output_cols=CATEGORICAL_COLUMNS_OE
                )
            ),
            (
                "MMS",
                MinMaxScaler(
                clip=True,
                input_cols=NUMERICAL_COLUMNS,
                output_cols=NUMERICAL_COLUMNS,
                )
            ),
            (
                "classification",
                SGDClassifier(
                input_cols=NUMERICAL_COLUMNS + CATEGORICAL_COLUMNS_OE + ['DEV_RECONNECTED'],
                label_cols=LABEL_COLUMNS,
                output_cols=OUTPUT_COLUMNS
                )
            )
    ]
)

#### Train the model
pipeline.fit below will internally create a stored procedure and run inside snowflake

In [11]:
pipeline.fit(train_df)

The version of package numpy in the local environment is 1.24.4, which does not fit the criteria for the requirement numpy==1.24.3. Your UDF might not work when the package version is different between the server and your local environment


In [12]:
result = pipeline.predict(test_df)

The version of package numpy in the local environment is 1.24.4, which does not fit the criteria for the requirement numpy==1.24.3. Your UDF might not work when the package version is different between the server and your local environment


In [20]:
y_true = sf_df.to_pandas()["FAILURE"]

In [22]:
from sklearn.metrics import confusion_matrix,log_loss,classification_report
report = classification_report(y_true,y_pred, output_dict=True)

/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [23]:
report

{'0': {'precision': 0.9989718545174142,
  'recall': 1.0,
  'f1-score': 0.9994856628519995,
  'support': 23319.0},
 '1': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 24.0},
 'accuracy': 0.9989718545174142,
 'macro avg': {'precision': 0.4994859272587071,
  'recall': 0.5,
  'f1-score': 0.49974283142599973,
  'support': 23343.0},
 'weighted avg': {'precision': 0.9979447661179618,
  'recall': 0.9989718545174142,
  'f1-score': 0.9984580461828289,
  'support': 23343.0}}

# Model Registry

In [24]:
from snowflake.snowpark import Session, Column, functions

In [25]:
REGISTRY_DATABASE_NAME = "MODEL_REGISTRY_SNOWFRACT"
REGISTRY_SCHEMA_NAME = "SNOWFRACT"

In [26]:
from snowflake.ml.registry import model_registry

In [28]:
# create session with account admin access
session = get_session()

In [29]:
model_registry.create_model_registry(session=session, database_name=REGISTRY_DATABASE_NAME, schema_name=REGISTRY_SCHEMA_NAME)

True

In [30]:
registry = model_registry.ModelRegistry(session=session, database_name=REGISTRY_DATABASE_NAME, schema_name=REGISTRY_SCHEMA_NAME)

In [31]:
registry.get_history().show()

-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"EVENT_TIMESTAMP"                 |"EVENT_ID"                        |"MODEL_ID"                        |"ROLE"          |"OPERATION"  |"ATTRIBUTE_NAME"  |"VALUE[ATTRIBUTE_NAME]"                             |
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|2023-08-03 11:37:47.031000+00:00  |2b1437f231f211eeab49c6770d61e4f3  |29da402031f211eeab49c6770d61e4f3  |"ACCOUNTADMIN"  |SET          |REGISTRATION      |{                                                   |
|                                  |                                  |                                  |                |             |                  |  "C

In [32]:
MODEL_NAME="PredictiveMaintenance"
MODEL_VERSION="hg3295ef16a2c0fe34d17f557600l45j"

In [33]:
# A name and model tags can be added to the model at registration time.
model_id = registry.log_model(
    model_name=MODEL_NAME,
    model_version=MODEL_VERSION,
    model=pipeline,
    tags={"stage": "testing", "classifier_type": "SGDClassifier"},
    sample_input_data=test_df.show(),
    options={"embed_local_ml_library": True}
)

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"FAILURE"  |"METRIC5"  |"METRIC6"  |"DAY"  |"DAY_WEEK"  |"SECTOR"  |"OP_PERIOD"  |"DEV_RECONNECTED"  |"MNW1"    |"DIF_M6"  |"DIF_M5"  |"LOG_M2"           |"LOG_M3"            |"LOG_M4"            |"LOG_M7"  |"LOG_M9"            |
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|0          |14         |311869     |1      |3           |S1F0      |1            |0                  |1027230   |0         |0         |0.0                |0.6931471805599453  |0.0                 |0.0       |0.0                 |
|0          |9          |389730     |1      |3           |S1F0      |1      

In [34]:
model_id

'3d6bcc1a577411eebfb12e186f38c059'

## List models form snowflake repository


In [35]:
model_list = registry.list_models()

In [36]:
model_list.to_pandas()

,CREATION_CONTEXT,CREATION_ENVIRONMENT_SPEC,CREATION_ROLE,CREATION_TIME,ID,INPUT_SPEC,NAME,OUTPUT_SPEC,RUNTIME_ENVIRONMENT_SPEC,TYPE,URI,VERSION,DESCRIPTION,METRICS,TAGS,REGISTRATION_TIMESTAMP
0,None,"{\n ""python"": ""3.8.12""\n}","""ACCOUNTADMIN""",2023-08-03 11:37:46.109000+00:00,29da402031f211eeab49c6770d61e4f3,None,ChurnPrediction,None,None,snowml,sfc:MODEL_REGISTRY_SNOWFRACT.SNOWFRACT.SNOWML_...,ee9215ef16a2c0fe34d17f557600a34b,None,"{\n ""classification_report"": ""NA""\n}","{\n ""classifier_type"": ""xgboost"",\n ""stage"":...",2023-08-03 11:37:47.031000+00:00
1,None,"{\n ""python"": ""3.8.12""\n}","""ACCOUNTADMIN""",2023-08-17 11:22:03.077000+00:00,495c6e9c3cf011ee8afe22df4b63f4a1,None,ChurnPrediction,None,None,snowml,sfc:MODEL_REGISTRY_SNOWFRACT.SNOWFRACT.SNOWML_...,ee9215ef16a2c0fe34d17f557600a35b,None,None,"{\n ""classifier_type"": ""xgboost"",\n ""stage"":...",2023-08-17 11:22:04.341000+00:00
2,None,"{\n ""python"": ""3.8.12""\n}","""ACCOUNTADMIN""",2023-08-21 07:19:55.837000+00:00,200e68963ff311ee89fa628aca6d24dc,None,ChurnPrediction,None,None,snowml,sfc:MODEL_REGISTRY_SNOWFRACT.SNOWFRACT.SNOWML_...,ee9215ef16a2c0fe34d17f557600a36c,None,"{\n ""build_time_metrics"": {\n ""accuracy"": ...","{\n ""classifier_type"": ""xgboost"",\n ""stage"":...",2023-08-21 07:19:57.339000+00:00
3,None,"{\n ""python"": ""3.8.16""\n}","""ACCOUNTADMIN""",2023-09-20 05:12:07.700000+00:00,3d6bcc1a577411eebfb12e186f38c059,None,PredictiveMaintenance,None,None,snowml,sfc:MODEL_REGISTRY_SNOWFRACT.SNOWFRACT.SNOWML_...,hg3295ef16a2c0fe34d17f557600l45j,None,None,"{\n ""classifier_type"": ""SGDClassifier"",\n ""s...",2023-09-20 05:12:09.618000+00:00


In [37]:
model_list.filter(model_list["ID"] == model_id).select("NAME","TAGS","METRICS").show()

-----------------------------------------------------------------------------
|"NAME"                 |"TAGS"                                 |"METRICS"  |
-----------------------------------------------------------------------------
|PredictiveMaintenance  |{                                      |NULL       |
|                       |  "classifier_type": "SGDClassifier",  |           |
|                       |  "stage": "testing"                   |           |
|                       |}                                      |           |
-----------------------------------------------------------------------------



## Adding metrics to a model registered

In [38]:
registry.set_metric(MODEL_NAME,MODEL_VERSION,"build_time_metrics", report)

In [39]:
model_list = registry.list_models()
model_list.filter(model_list["ID"] == model_id).select("NAME","TAGS","METRICS").show()

----------------------------------------------------------------------------------------------------------
|"NAME"                 |"TAGS"                                 |"METRICS"                               |
----------------------------------------------------------------------------------------------------------
|PredictiveMaintenance  |{                                      |{                                       |
|                       |  "classifier_type": "SGDClassifier",  |  "build_time_metrics": {               |
|                       |  "stage": "testing"                   |    "0": {                              |
|                       |}                                      |      "f1-score": 0.9994856628519995,   |
|                       |                                       |      "precision": 0.9989718545174142,  |
|                       |                                       |      "recall": 1,                      |
|                       |            

### Load model from snowflake repo

In [41]:
model_list = registry.list_models()

In [42]:
ppln_loaded = model_registry.ModelReference(registry=registry, model_name=MODEL_NAME, model_version=MODEL_VERSION)

In [44]:
type(ppln_loaded)

snowflake.ml.registry.model_registry.ModelReference

In [43]:
# need model id for this
ppln_loaded.deploy(
    deployment_name="pred_maintenance_final",
    target_method="predict",
    permanent=True,
    options={"relax_version":True}
)

/opt/conda/lib/python3.8/site-packages/snowflake/ml/model/_deploy_client/warehouse/deploy.py:178: RuntimeWarning: Cannot find conda resolver, use Snowflake information schema for best-effort dependency pre-check.
  warnings.warn(


Generated UDF file is persisted at: /tmp/tmp2kzabxun.py


"MODEL_REGISTRY_SNOWFRACT"."SNOWFRACT"."pred_maintenance_final" is deployed to warehouse.
